In [2]:
def parseInstructions(instructions):
    moves = 0
    for i, c in enumerate(instructions):
        if c.isdigit():
            moves = moves * 10 + int(c)
        else:
            yield moves
            moves = 0
            yield c
    if moves > 0:
        yield moves


def boundaries(board):
    n, m = len(board), len(board[0])
    rowBoundaries = [None] * n
    columnBoundaries = [None] * m
    for i, row in enumerate(board):
        start = 0
        for j in range(m):
            if row[j] != " ":
                start = j
                break
        end = m - 1
        for j in range(m - 1, -1, -1):
            if row[j] != " ":
                end = j
                break
        rowBoundaries[i] = (start, end)

    for j in range(m):
        start = 0
        for i in range(n):
            if board[i][j] != " ":
                start = i
                break
        end = n - 1
        for i in range(n - 1, -1, -1):
            if board[i][j] != " ":
                end = i
                break
        columnBoundaries[j] = (start, end)
    return rowBoundaries, columnBoundaries


def finalPassword(board, instructions):
    rowBoundaries, columnBoundaries = boundaries(board)
    # 0 for right (>), 1 for down (v), 2 for left (<), and 3 for up (^)
    step = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    facing = 0
    x, y = 0, rowBoundaries[0][0]
    for instruction in instructions:
        if instruction == "R":
            facing = (facing + 1) % 4
        elif instruction == "L":
            facing = (facing - 1) % 4
        else:
            for i in range(instruction):
                xn = x + step[facing][0]
                yn = y + step[facing][1]
                if facing % 2 == 0:  # horizontal
                    if yn == rowBoundaries[xn][1] + 1:
                        yn = rowBoundaries[xn][0]
                    elif yn == rowBoundaries[xn][0] - 1:
                        yn = rowBoundaries[xn][1]
                else:  # vertical
                    if xn == columnBoundaries[yn][1] + 1:
                        xn = columnBoundaries[yn][0]
                    elif xn == columnBoundaries[yn][0] - 1:
                        xn = columnBoundaries[yn][1]
                if board[xn][yn] == "#":
                    break  # continue with next instruction
                x, y = xn, yn
    print(x, y, facing, 1000 * (x + 1) + 4 * (y + 1) + facing)


with open("22.input", "r") as f:
    data, instructions = f.read().split("\n\n")
    board = data.splitlines()
    m = max(len(row) for row in board)
    for i, row in enumerate(board):
        if len(row) < m:
            board[i] = row + " " * (m - len(row))
    instructions = parseInstructions(instructions.rstrip())
    finalPassword(board, instructions)


59 89 2 60362


In [17]:
"""
22.input:
   [0][1]
   [2]
[3][4]
[5]
"""

useSample = False
inputFile = "22.sample" if useSample else "22.input"

faceWidth = 50
faceBoundaries = [
    (0, 50, 49, 99),
    (0, 100, 49, 149),
    (50, 50, 99, 99),
    (100, 0, 149, 49),
    (100, 50, 149, 99),
    (150, 0, 199, 49),
]

if useSample:
    faceWidth = 4
    faceBoundaries = [
        (0, 8, 3, 11),
        (4, 0, 7, 3),
        (4, 4, 7, 7),
        (4, 8, 7, 11),
        (8, 8, 11, 11),
        (8, 12, 11, 15),
    ]


def wrapping(faceIndex, facing, r, c):
    """
    wrapping rules for the 6 cube faces.
    facing is 0 for right (>), 1 for down (v), 2 for left (<), and 3 for up (^).
    """
    wrappingRules = [
        [(1, 0, r, 100), (2, 1, 50, c), (3, 0, 149 - r, 0), (5, 0, 100 + c, 0)],
        [(4, 2, 149 - r, 99), (2, 2, c - 50, 99), (0, 2, r, 99), (5, 3, 199, c - 100)],
        [(1, 3, 49, r + 50), (4, 1, 100, c), (3, 1, 100, r - 50), (0, 3, 49, c)],
        [(4, 0, r, 50), (5, 1, 150, c), (0, 0, 149 - r, 50), (2, 0, c + 50, 50)],
        [(1, 2, 149 - r, 149), (5, 2, c + 100, 49), (3, 2, r, 49), (2, 3, 99, c)],
        [(4, 3, 149, r - 100), (1, 1, 0, 100 + c), (0, 1, 0, r - 100), (3, 3, 149, c)],
    ]
    if useSample:
        wrappingRules = [
            [(5, 2, 11 - r, 15), (3, 1, 4, c), (2, 1, 4, 4 + r), (1, 1, 4, 11 - c)],
            [(2, 0, r, 4), (4, 3, 11, 11 - c), (5, 3, 11, 19 - r), (0, 1, 0, 11 - c)],
            [(3, 0, r, 8), (4, 0, 15 - c, 8), (1, 2, r, 3), (0, 0, c - 4, 8)],
            [(5, 1, 8, 19 - r), (4, 1, 8, c), (2, 2, r, 7), (0, 3, 3, c)],
            [(5, 0, r, 12), (1, 3, 7, 11 - c), (2, 3, 7, 15 - r), (3, 3, 7, c)],
            [(0, 2, 11 - r, 11), (1, 0, 19 - c, 0), (4, 2, r, 11), (3, 2, 19 - c, 11)],
        ]
    return wrappingRules[faceIndex][facing]


def cubePath(board, instructions, r, c):
    facing = 0
    faceIndex = 0
    step = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    n, m = len(board), len(board[0])
    for instruction in instructions:
        if instruction == "R":
            facing = (facing + 1) % 4
        elif instruction == "L":
            facing = (facing - 1) % 4
        else:
            for i in range(instruction):
                rn, cn = r + step[facing][0], c + step[facing][1]
                minRow, minCol, maxRow, maxCol = faceBoundaries[faceIndex]
                # in the same face
                if minRow <= rn <= maxRow and minCol <= cn <= maxCol:
                    if board[rn][cn] == "#":
                        break  # continue with next instruction
                    r, c = rn, cn
                else:
                    newFaceIndex, newFacing, newR, newC = wrapping(
                        faceIndex, facing, r, c
                    )
                    # print(newFaceIndex, newFacing, newR, newC)
                    # assert faceBoundaries[newFaceIndex][0] <= newR <= faceBoundaries[newFaceIndex][2] and faceBoundaries[newFaceIndex][1] <= newC <= faceBoundaries[newFaceIndex][3]
                    if board[newR][newC] == "#":
                        break  # continue with next instruction
                    faceIndex, facing, r, c = newFaceIndex, newFacing, newR, newC

    print(r, c, facing, 1000 * (r + 1) + 4 * (c + 1) + facing)


with open(inputFile, "r") as f:
    data, instructions = f.read().split("\n\n")
    board = data.splitlines()
    m = max(len(row) for row in board)
    for i, row in enumerate(board):
        if len(row) < m:
            board[i] = row + " " * (m - len(row))
    instructions = parseInstructions(instructions.rstrip())
    cubePath(
        board, instructions, 0, 2 * faceWidth if inputFile == "22.sample" else faceWidth
    )


73 71 0 74288
